<a href="https://colab.research.google.com/github/05-515/Stepik_ChatGPT/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%223_2_%D0%A0%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai langchain-openai langchain-community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9

In [2]:
import re
import pandas as pd
from tqdm import tqdm
from getpass import getpass

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, TransformChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [ ]:
import os
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [3]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2024-09-17 12:02:53--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11316 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.05K  --.-KB/s    in 0s      

2024-09-17 12:02:53 (70.4 MB/s) - ‘utils.py’ saved [11316/11316]



In [4]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш API ключ, полученный в боте курса··········


## Задание 3.2.9 🤔 Кажется, это что-то на LLM-ском? 🧐

In [5]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
df.head()

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."


Напишем функцию, которая очистит текст от ненужных символов: `¿, ¡, £`

In [12]:
df.raw_text[0]

"The sun was setting, casting long shadows over the small town. Jo¿hn, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same tow¡n, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they s¡at there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they looked at the setting sun, they realized that those days were long gone. John, however, was not ready to give up. He still had dreams, dreams that he was determined to fulf

In [62]:
def clean_text(inputs):
    #text = inputs["raw_text"]

    # Удаляем странные символы
    cleaned_text = re.sub(r'[¿¡£]', '', inputs)

    return cleaned_text


#print(clean_text(result.raw_text[0]))

Будем просить у модели определять язык и имя главного персонажа и выдавать ответ в виде словаря. Для этого создадим `Output parser`, с которым вы уже познакомились в прошлых уроках.

In [22]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
# Определим схемы ответа
language_schema = ResponseSchema(name="language",
                             description="Определи язык, на котором написан текст? название языка напиши на английском языке")

person_schema = ResponseSchema(name="main_character",
                             description="Определи имя главного персонажа в тексте. Напиши на том языке, на котором и написан сам текст")

response_schemas = [language_schema, person_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas) # Создаём парсер и подаём в него список со схемами
format_instructions = output_parser.get_format_instructions() # Получаем инструкции по форматированию ответа

In [23]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"language": string  // Определи язык, на котором написан текст? название языка напиши на английском языке
	"main_character": string  // Определи имя главного персонажа в тексте. Напиши на том языке, на котором и написан сам текст
}
```


Напишем шаблон промпта со своим вопросом и инструкциями по форматированию ответа. Будем передавать в этот промпт сырой текст

In [13]:
from langchain.prompts import ChatPromptTemplate

prompt_template = """ Из следующего текста извлеки информацию:

language: Определи язык, на котором написан текст. название языка напиши на английском языке

main_character: Определи имя главного персонажа в тексте. Напиши на том языке, на котором и написан сам текст

выведите их в виде списка Python, разделенного запятыми.

text: {text}

{format_instructions}

"""
prompt = PromptTemplate(input_variables=['text', 'format_instructions'], template=prompt_template)

In [14]:
print(messages[0].content)

 Из следующего текста извлеки информацию:

language: БОпредели язык, на котором написан текст? название языка напиши на английском языке

person: Определи имя главного персонажа в тексте. Напиши на том языке, на котором и написан сам текст

выведите их в виде списка Python, разделенного запятыми.

text: The sun was setting, casting long shadows over the small town. Jo¿hn, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same tow¡n, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they s¡at there, reminiscing about the past, a sense of nostalgia washed over 

In [16]:
chat = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)
response = chat.invoke(messages)

In [17]:
print(response.content)

```json
{
	"language": "English",
	"person": "John"
}
```


In [20]:
output_dict = output_parser.parse(response.content)
type(output_dict)
output_dict.get('language')

'English'

Создадим цепочку с помощью `LCEL`

In [ ]:
chain = prompt | llm # И ВСЁ!

In [70]:
result = pd.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
result.head()

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."


In [71]:
results = []
for text in tqdm(result['raw_text']):
    answer = chain.invoke({'text': text, 'format_instructions': format_instructions})
    output_dict = output_parser.parse(answer.content)
    results.append({
            'text': clean_text(text),
            'language': output_dict.get('language'),
            'main_character': output_dict.get('main_character')
    })
    #result.at[index, 'text'] = clean_text(result['raw_text'])
    #result.at[index, 'language'] = answer.content('language')
    #result.at[index, 'main_character'] = answer.content('main_character')
    #break # Для отладки. Уберите, когда убедитесь, что на одном примере работает
result = pd.DataFrame(results)
print(result)

100%|██████████| 13/13 [00:12<00:00,  1.03it/s]

                                                 text language main_character
0   The sun was setting, casting long shadows over...  English           John
1   Le soleil se couchait, jetant de longues ombre...   French         Pierre
2   El sol se estaba poniendo, proyectando largas ...  Spanish         Carlos
3   La ciudad estaba llena de vida, sus calles lle...  Spanish           Juan
4   La ville était pleine de vie, ses rues remplie...   French           Jean
5   Die Stadt war voller Leben, ihre Straßen gefül...   German         Johann
6   Die Sonne ging unter und warf lange Schatten ü...   German           Hans
7   В тихом уголке старого города, где узкие улочк...  Russian           Анна
8   In a small town nestled between the mountains ...  English          Laura
9   En un pequeño pueblo situado entre las montaña...  Spanish          Maria
10  Dans un petit village niché entre les montagne...   French         Sophie
11  В городе было множество людей, каждый из котор...  Russian  

In [25]:
print(answer.content)

```json
{
	"language": "English",
	"main_character": "John"
}
```


Сохраним всё в итоговый файл. Убедитесь, что на этом этапе у вас в столбцах

- `text` - очищенный текст (без символов ¿, ¡, £)
- `language` - язык, на котором написан текст (название языка указать на английском языке)
- `main_character` - имя главного персонажа в тексте (указать на том языке, на котором и написан сам текст)

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:
result.to_csv('/content/drive/My Drive/3.2.9_solution.csv', index=False)